# Tugas 1 : Crawling Berita

## Adz Dzikry Pradana Putra - 210411100093
## Penambangan Web A

### 1. Import library yang dibutuhkan untuk melakukan web crawl

* Menggunakan BeautifulSoup, konten HTML dari halaman web diurai untuk mencari elemen artikel yang relevan.

* Menggunakan pandas Library digunakan untuk membuat dan mengelola DataFrame.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

### 2. Inisialisasi Data

Membuat Variable kosong yang berfungsi untuk menyimpan data yang diambil dari berita

In [2]:
# Mendefinisikan list untuk menyimpan data yang diambil
judul = []
isi = []
tanggal = []
url_list = []
kategori_list = []

### 3. Membuat fungsi filter konten

Berfungsi untuk Mem-filter dari elemen-elemen HTML pada berita yang tidak diinginkan , Contoh kasus seperti iklan, daftar isi, gambar , link sisipan, dll

In [3]:
# Fungsi untuk membersihkan konten dari elemen-elemen yang tidak diinginkan
def clean_content(content_element):
    if content_element:
        # Hapus elemen yang berisi daftar isi
        for daftar_isi in ["collapsible"]:
            unwanted = content_element.find("div", id=daftar_isi)
            if unwanted:
                unwanted.decompose()

        # Hapus elemen yang berisi tag
        for tag_class in ["aevp", "detail__body-tag mgt-16"]:
            unwanted = content_element.find_all("div", class_=tag_class)
            for el in unwanted:
                el.decompose()

        # Hapus elemen yang berisi link sisipan
        link_sisip = content_element.find_all("table", class_="linksisip")
        for table in link_sisip:
            table.decompose()

        # Hapus elemen paragraf dan span dengan class 'para_caption'
        unwanted_paragraphs = content_element.find_all(["p", "span"], class_="para_caption")
        for para in unwanted_paragraphs:
            para.decompose()

        # Kembalikan teks yang tersisa
        return content_element.get_text(separator=' ', strip=True).strip()

    return "Content Not Found"

### 4. Membuat fungsi untuk melakukan crawling data pada situs web Detik.com.

Fungsi ini mengambil data berupa judul berita, tanggal publikasi, isi berita dan kategori berita yang terdapat di halaman tersebut.



* mencari semua elemen-elemen yang mengandung isi penting pada artikel seperti elemen yang berisi judul, tanggal, isi dan kategori berita
* Judul, tanggal, dan isi artikel yang valid kemudian disimpan ke dalam list judul, tanggal, isi dan kategori berita.

In [4]:
# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, max_articles):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len([k for k in kategori_list if k == kategori]) >= max_articles:
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")

        # Bersihkan konten menggunakan fungsi clean_content
        content = clean_content(content_element)

        judul.append(title)
        tanggal.append(date)
        isi.append(content)
        url_list.append(link)
        kategori_list.append(kategori)

### 5. Inisialisasi URL dan Kategori yang Akan Di-Crawl

* max_article = variable ini digunakan sebagai batas maksimum berita yang akan diambil

In [5]:
# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://inet.detik.com/games/indeks","https://sport.detik.com/sepakbola/indeks"]
categories = ["Games","Sepak Bola"]

max_articles = 10

### 6. Melakukan Iterasi untuk Setiap URL dan Kategori yang akan di Crawl

In [6]:
# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < max_articles:
        url = f"{base_url}/{page}"
        get_data(url, category, max_articles)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman
        page += 1

### 7. Menampilkan Hasil Crawl dengan dataframe

Menyimpan hasil Crawl ke CSV , lalu menampilkan hasil Crawl menggunakan Dataframe

In [7]:
# Membuat dataframe dari list data
df1 = pd.DataFrame({"judul": judul, "isi": isi, "tanggal": tanggal, "kategori": kategori_list})

# Menyimpan dataframe ke file csv
df1.to_csv("data_berita_detik.csv", index=False)
df=pd.read_csv("data_berita_detik.csv")
df

,judul,isi,tanggal,kategori
0,3 Cara Log Out Akun ML 2024 dan Beralih Akun Baru,Jakarta - Mobile Legends (ML) adalah salah sat...,"Sabtu, 07 Sep 2024 19:41 WIB",Games
1,Ada Apa dengan Indostars di FFWS SEA 2024 Fall...,Jakarta - Free Fire World Series (FFWS) Southe...,"Sabtu, 07 Sep 2024 17:18 WIB",Games
2,"NBA 2K25 Rilis di PS4, PS5, Xbox, dan PC, Segi...",Jakarta - 2K selaku penerbit dan Visual Concep...,"Sabtu, 07 Sep 2024 16:14 WIB",Games
3,Film Adaptasi Game Minecraft Rilis Teaser Trai...,Jakarta - Warner Bros Pictures telah merilis t...,"Sabtu, 07 Sep 2024 11:30 WIB",Games
4,Universitas Terbuka Gelar Turnamen Mobile Lege...,Jakarta - Universitas Terbuka (UT) menggelar M...,"Jumat, 06 Sep 2024 22:14 WIB",Games
5,Tips Jago Football Manager 2024 Ala Juara Duni...,"Jakarta - Tahukah detikers, Indonesia sukses m...","Jumat, 06 Sep 2024 21:40 WIB",Games
6,10 Emulator Android Ringan 2024 untuk PC atau ...,Jakarta - Ternyata ada lho emulator Android ya...,"Jumat, 06 Sep 2024 20:40 WIB",Games
7,Iklan Game Hero Wars di X Sekarang Dikasih 'Pe...,Jakarta - Ketika gamer menonton YouTube atau m...,"Jumat, 06 Sep 2024 17:40 WIB",Games
8,7 Tim Indonesia Lolos Final 2024 PMSL SEA Fall...,Jakarta - Babak League Stage 2024 PMSL SEA Fal...,"Jumat, 06 Sep 2024 15:06 WIB",Games
9,Jadwal MPL ID S14 Week 5 Hari Ini: Evos Glory ...,Jakarta - Jadwal MPL ID S14 pekan kelima disel...,"Jumat, 06 Sep 2024 12:15 WIB",Games
